In [0]:
appId = dbutils.secrets.get(scope = "ADXappIP", key = "ADBappIP") 
appSecret = dbutils.secrets.get(scope = "ADXappSecret", key = "ADBappSecret") 
tenantId = dbutils.secrets.get(scope = "tenantId", key = "tenantId") 

configs = {"fs.azure.account.auth.type": "OAuth",
        "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
        "fs.azure.account.oauth2.client.id": appId,
        "fs.azure.account.oauth2.client.secret": appSecret,
        "fs.azure.account.oauth2.client.endpoint":  f"https://login.microsoftonline.com/{tenantId}/oauth2/token"}

In [0]:
def sub_mount(str_path, str_mountPoint):
    if any(mount.mountPoint == str_mountPoint for mount in dbutils.fs.mounts()):
      print( "--")
    else:
      try:
        dbutils.fs.mount(
          source = str_path,
          mount_point = str_mountPoint,
          extra_configs = configs
        )
      except Exception as e:
        print(f"ERROR: {mountPoint} error.")
        
def sub_unmount(str_path):
  if any(mount.mountPoint == str_path for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(str_path)

def mount_exists( mp: str ):

  returnValue = False

  print( "Mounting: " + mp )
  for m in dbutils.fs.mounts():
      mountExists = (m.mountPoint==mp)
      if mountExists:
          returnValue = True
          break

  return returnValue


In [0]:
def exec_mount_container( containerName: str, storageAccountName: str ):

    """ Establishes a mount point to /mnt

        Parameters:  containerName:  The name of the Azure Data Lake Storage container passed in as a string
                     storageAccountName:  The Azure Data Lake storage account name passed in as a string

        Example:  mountPoint = exec_mount_container( "myContainerName", "myStorageAccountName" )             

        Returns:  The mount point
    """

    mountPoint = f"/mnt/{containerName}"

    if mount_exists( mountPoint ):
        print(f"Mount point {mountPoint} already exists")

    else:
        str_path = 'abfss://{}'.format(containerName)+'@'+'{}.dfs.core.windows.net/'.format(storageAccountName)
        print(f"Creating mount point {mountPoint} for " + str_path)
        sub_mount(str_path, mountPoint)

    return mountPoint